# UMAP for bird-songs  
---

In [ ]:
## Installs for Coursera ^^
# conda install -c conda-forge librosa umap-learn
# conda install -c plotly plotly=4.14.3

In [3]:
## Installs for CoLab
!pip install umap-learn
!pip install plotly==4.14.3

     |████████████████████████████████| 81kB 5.8MB/s 
     |████████████████████████████████| 1.2MB 21.4MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.1-cp37-none-any.whl size=76569 sha256=dd899c38846b390bd1d19571e0f5329210ee00ed1dcf40b504b1563b5a657595
  Stored in directory: /root/.cache/pip/wheels/ad/df/d5/a3691296ff779f25cd1cf415a3af954b987fb53111e3392cf4
  Created wheel for pynndescent: filename=pynndescent-0.5.2-cp37-none-any.whl size=51351 sha256=3e68939ce2792732e650f7645ffcd190179f617cda5a87a43e59970179347496
  Stored in directory: /root/.cache/pip/wheels/ba/52/4e/4c28d04d144a28f89e2575fb63628df6e6d49b56c5ddd0c74e
Successfully built umap-learn pynndescent


In [1]:
# For Colab only
# 4/1AY0e-g6CTocvBK45PWlJu2ycBuOMTgu36b-VZgpnCXitW_Vy3ckpGGBzur8
from google.colab import drive
drive.mount('content')
# /content/content/MyDrive/bird-songs/audio

Drive already mounted at content; to attempt to forcibly remount, call drive.mount("content", force_remount=True).


In [1]:
from umap import UMAP
from sklearn.datasets import load_digits
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'iframe'

import os
import numpy as np
from matplotlib import pyplot as plt
import IPython.display as ipd
import librosa
import librosa.display
import pandas as pd
import glob
# import ffmpeg
%matplotlib inline

# Set general font size
plt.rcParams['font.size'] = '14'

import warnings
warnings.filterwarnings("ignore")

from tqdm.notebook import tqdm


In [2]:
# Check whether google drive mounted
path = '/content/content/MyDrive/bird-songs/'
if os.path.isdir(path) == True:
    print('Google Drive Mounted')
    run_on_colab = True
else:
    print('Using local drive')
    run_on_colab = False
    

Using local drive


In [5]:
#@title
# def get_audio_filenames(audio_folder):
#     '''Create a list of audio files in the provided folder'''
    
#     audio_files = []
#     for fn_mp3 in glob.glob(f"{audio_folder}*"):
#         audio_files.append(fn_mp3)
#     return audio_files

# if run_on_colab ==True:
#     audio_filenames = get_audio_filenames('/content/content/MyDrive/bird-songs/audio/')
#     print(f'{len(audio_filenames)} audio files appended to list from Google Drive')
# else:
#     audio_filenames = get_audio_filenames('./')
#     print(f'{len(audio_filenames)} audio files appended to list from local drive')


0 audio files appended to list from Google Drive


In [6]:
if run_on_colab == True:
    features = pd.read_csv('/content/content/MyDrive/bird-songs/features_filtered.csv', encoding='latin')
else:
    features = pd.read_csv('features_filtered.csv', encoding='latin')
# display (features.head())

# These are the top species we selected for our analysis
selected_species = ['Red-winged Blackbird', 
                    'Common Yellowthroat', 
                    'Northern Cardinal', 
                    'Carolina Wren', 
                    'Red Crossbill', 
                    'Spotted Towhee']
# We only need the id and the labels for our CNN.
dataset = features[features.en.isin(selected_species)][['id', 'en']]
# Now we need to change our labels into integers as torch does not support string format for labels.
labels = {"Red-winged Blackbird": 0, 'Common Yellowthroat': 1, 'Northern Cardinal': 2, 'Carolina Wren': 3,
          'Red Crossbill': 4, 'Spotted Towhee': 5}
dataset['label'] = dataset.apply(lambda row: labels[row.en], axis=1)
dataset.drop('en', axis=1, inplace=True)
dataset.set_index('id', inplace=True)
display (dataset)

audio_filenames = dataset.index.tolist()

if run_on_colab == True:
    audio_filenames = list(map(lambda x: '/content/content/MyDrive/bird-songs/audio_8sec/' + str(x) + '.wav',
                            audio_filenames))
else:
    audio_filenames = list(map(lambda x: '/audio_8sec/' + str(x) + '.wav', audio_filenames))

len(audio_filenames)

,label
id,
564895,3
545775,3
540857,3
540855,3
539275,3
...,...
233717,2
216628,2
172627,2


2516

In [10]:
#@title
# def create_df_xeno_canto(csv_file, audio_folder, min_dur, max_dur):
#     '''Create pandas dataframe with features provided by xeno-canto and append file name of audio file'''
    
#     df = pd.read_csv(csv_file, header=None)
#     print(df.head())
#     df.columns = ['id', 'gen', 'sp', 'ssp', 'en', 'cnt', 'loc','type', 'q', 'length', 'bird-seen', 'link']
#     if run_on_colab ==True:
#         df['filename'] = df['id'].map('/content/content/MyDrive/bird-songs/audio/{}.mp3'.format, na_action='ignore')
#     else:
#         df['filename'] = df['id'].map('audio/{}.mp3'.format, na_action='ignore')
    
#     df['seconds'] = df['length'].apply(lambda x: int(x[0])*60 + int(x[-2:]))
    
#     df = df[(df['seconds']>=min_dur) & (df['seconds']<=max_dur)]
    
#     df.set_index('id', inplace=True)

#     return df


# if run_on_colab ==True:
#     df_xeno_canto = create_df_xeno_canto('/content/content/MyDrive/bird-songs/features.csv', 'audio/', 2, 10)
# else:
#     df_xeno_canto = create_df_xeno_canto('features.csv', 'audio/', 2, 10)


# # df_xeno_canto.sort_values(by='seconds', ascending=True).head()
# df_xeno_canto

In [7]:
def create_mfcc_spectral_features(number_mfcc, sample_rate=22050):
    ''' '''

    # Create df to hold mfcc spectral features
    df = pd.DataFrame(columns = ['mfcc_avg' + str(item+1) for item in list(range(number_mfcc))] +
                                ['mfcc_std' + str(item+1) for item in list(range(number_mfcc))], 
                                index=dataset.index)
        
    for idx, audio_filename in tqdm(enumerate(audio_filenames)):

        y, sr = librosa.load(audio_filename, sr=sample_rate, mono=True)
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=number_mfcc, n_fft=2048, hop_length=1024, fmin=500)
        # print(mfccs.shape)

        mfcc_means = [np.mean(item) for item in mfccs]
        mfcc_stds = [np.std(item) for item in mfccs]
        # print("mfcc means:", mfcc_means, "\nmfcc std deviations:", mfcc_stds)
        df.iloc[idx] = mfcc_means + mfcc_stds
  

    return df


number_mfcc = 24
df_mfcc = create_mfcc_spectral_features(number_mfcc)
df_mfcc

,mfcc_avg1,mfcc_avg2,mfcc_avg3,mfcc_avg4,mfcc_avg5,mfcc_avg6,mfcc_avg7,mfcc_avg8,mfcc_avg9,mfcc_avg10,mfcc_avg11,mfcc_avg12,mfcc_avg13,mfcc_avg14,mfcc_avg15,mfcc_avg16,mfcc_avg17,mfcc_avg18,mfcc_avg19,mfcc_avg20,mfcc_avg21,mfcc_avg22,mfcc_avg23,mfcc_avg24,mfcc_std1,mfcc_std2,mfcc_std3,mfcc_std4,mfcc_std5,mfcc_std6,mfcc_std7,mfcc_std8,mfcc_std9,mfcc_std10,mfcc_std11,mfcc_std12,mfcc_std13,mfcc_std14,mfcc_std15,mfcc_std16,mfcc_std17,mfcc_std18,mfcc_std19,mfcc_std20,mfcc_std21,mfcc_std22,mfcc_std23,mfcc_std24
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
564895,-889.492,5.0335,-31.4773,4.73073,22.9555,-11.6972,-14.5283,10.5064,-0.979624,-6.92005,3.75479,5.51634,-1.98331,4.45234,1.58094,-6.79015,0.780834,4.96603,-3.84318,-3.38185,3.86442,0.416703,-3.63969,1.73831,54.8656,20.6248,61.4001,23.5236,43.6314,28.4721,24.2452,27.3742,11.3261,21.4315,12.8862,15.7766,11.2154,11.5864,7.85349,13.7414,10.8817,13.568,11.2003,6.76007,8.62615,6.70962,8.03866,7.58892
545775,-629.473,-44.4655,-47.775,61.9831,-1.03335,-0.152016,-10.5474,2.96929,1.58006,10.9912,4.00956,-1.31302,3.10983,0.962198,-0.161864,0.784014,-1.68879,-7.63529,3.80711,4.49119,-2.39815,-2.61737,2.86433,1.15802,144.486,76.5706,76.4662,70.7324,20.5744,22.7741,32.5819,14.9177,13.7567,20.6891,17.6878,13.1438,12.5093,13.144,15.8983,14.3398,12.4027,17.6698,11.539,14.7047,12.2775,9.80263,10.7511,9.22824
540857,-837.42,20.6867,-56.4215,-14.9488,2.62786,-2.47709,-3.9447,-4.24075,-2.73174,-1.02478,6.68777,-0.485857,-1.94842,13.7119,6.50528,-4.9148,-3.66503,-2.16639,-1.29081,-3.2789,-4.2712,-1.3895,4.98414,2.9093,69.8873,41.4715,57.9993,51.3175,43.4628,34.1108,25.187,30.4362,18.6808,20.153,17.7549,20.3741,18.7521,20.6494,18.654,17.1558,16.8182,18.3447,16.3874,14.9901,17.5711,12.5783,16.3291,12.3219
540855,-812.35,-12.1207,-45.2474,3.26925,11.5764,1.21442,-3.43979,4.12308,-5.66603,5.67809,-3.74287,-0.625781,4.12183,-4.8242,1.30128,-1.2018,7.83683,0.461579,-8.2654,1.14232,0.390527,0.0296896,0.416522,4.08837,83.4465,28.7416,80.8335,35.1697,39.1032,27.3928,19.5737,21.4965,18.8731,12.5815,14.7667,14.2993,13.5988,13.3202,17.7982,12.3377,14.5337,14.1897,12.7729,11.4186,10.0065,12.535,8.72309,10.5673
539275,-809.427,-13.6542,-66.897,20.6821,26.0897,-18.9203,-4.99208,6.63673,2.71826,1.82983,-3.65343,1.77004,-1.35147,1.41414,6.57911,-2.67363,-9.17468,7.72575,-0.599251,-4.86938,3.05944,5.43887,-4.94199,-0.432073,67.8921,25.3258,59.1801,34.1383,31.0189,29.6407,23.0668,23.2403,20.7804,16.9785,22.8823,11.2935,18.1583,12.5242,14.9922,13.694,15.1388,14.7116,11.8546,12.5495,9.18288,12.2467,8.50237,11.9813
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233717,-644.563,-22.3604,-87.1794,-25.6538,73.1685,22.7612,-37.2544,-0.544135,2.00335,1.07039,-6.57756,20.8402,-10.544,-8.32412,-9.40841,18.186,-12.5556,0.218724,7.97618,-8.50545,-7.90428,6.24422,1.8804,-1.35027,106.076,33.068,72.0949,31.421,66.3103,28.0327,39.9547,21.4171,22.4486,17.069,15.2844,25.2782,17.6313,13.6884,18.2959,18.7989,15.7396,11.5169,11.1462,14.2452,9.03244,10.3356,7.71181,6.76783
216628,-814.401,-7.16011,-54.5395,-27.4123,17.5851,19.2322,1.55713,-4.23501,-12.513,10.0558,1.93658,2.09161,-6.99099,1.58774,-5.97885,-1.62784,7.74817,6.61884,-2.09605,-5.39499,-0.841881,3.47881,-5.14781,3.97782,81.5493,38.7855,58.9721,47.8085,55.9656,44.5863,32.108,28.9185,24.6301,23.4071,18.8241,18.7953,18.1484,17.9966,14.8599,16.0634,15.9573,15.1945,9.98685,10.5553,10.7141,9.17735,7.59046,9.2421
172627,-731.129,-19.621,-34.9396,25.9774,7.6469,-11.7228,-2.42987,4.77666,4.65546,-6.38355,-1.1766,4.20618,-2.59369,2.04934,-0.725638,-1.51362,2.72227,-1.53539,-2.54818,4.53439,-1.89795,0.130091,-0.368581,-0.562726,55.7814,31.8759,52.3262,46.6176,32.4235,34.8767,25.4218,24.6472,25.0907,23.7248,21.0121,20.2428,14.4846,15.308,12.9771,13.7464,13.0285,13.2844,11.146,11.935,10.0525,10.1299,11.5871,8.12991


In [8]:
def create_mel_spectral_features(n_mels=128, sample_rate=22050):
    ''' '''

    # Create df to hold mfcc spectral features
    df = pd.DataFrame(columns = ['mel_avg' + str(item+1) for item in list(range(128))] +
                                ['mel_std' + str(item+1) for item in list(range(128))], 
                                index=dataset.index)
        

    for idx, audio_filename in tqdm(enumerate(audio_filenames)):
        

        y, sr = librosa.load(audio_filename, sr=sample_rate, mono=True)
        mels = librosa.feature.melspectrogram(y=y, sr=sample_rate, n_mels=n_mels, n_fft=2048, hop_length=1024, fmin=500)
        # print(mels.shape)
        mels_means = [np.mean(item) for item in mels]
        mels_stds = [np.std(item) for item in mels]
        # print("mel means:", mels_means, "\nmel std deviations:", mels_stds)
        df.iloc[idx] = mels_means + mels_stds
  
    return df


df_mel = create_mel_spectral_features()
df_mel

,mel_avg1,mel_avg2,mel_avg3,mel_avg4,mel_avg5,mel_avg6,mel_avg7,mel_avg8,mel_avg9,mel_avg10,mel_avg11,mel_avg12,mel_avg13,mel_avg14,mel_avg15,mel_avg16,mel_avg17,mel_avg18,mel_avg19,mel_avg20,mel_avg21,mel_avg22,mel_avg23,mel_avg24,mel_avg25,mel_avg26,mel_avg27,mel_avg28,mel_avg29,mel_avg30,mel_avg31,mel_avg32,mel_avg33,mel_avg34,mel_avg35,mel_avg36,mel_avg37,mel_avg38,mel_avg39,mel_avg40,...,mel_std89,mel_std90,mel_std91,mel_std92,mel_std93,mel_std94,mel_std95,mel_std96,mel_std97,mel_std98,mel_std99,mel_std100,mel_std101,mel_std102,mel_std103,mel_std104,mel_std105,mel_std106,mel_std107,mel_std108,mel_std109,mel_std110,mel_std111,mel_std112,mel_std113,mel_std114,mel_std115,mel_std116,mel_std117,mel_std118,mel_std119,mel_std120,mel_std121,mel_std122,mel_std123,mel_std124,mel_std125,mel_std126,mel_std127,mel_std128
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
564895,1.39991e-07,1.17269e-07,4.26156e-08,1.13841e-08,2.2498e-08,6.00953e-08,5.26074e-08,2.1328e-08,1.90172e-08,2.5101e-08,3.25694e-08,2.7357e-08,5.83439e-08,1.15998e-07,1.2304e-07,6.06394e-08,1.48074e-07,3.18107e-07,8.30244e-07,2.15034e-06,3.49189e-06,3.14802e-06,2.59008e-06,1.44405e-05,3.70047e-05,7.96621e-06,8.33104e-07,2.55496e-07,5.28958e-08,8.42188e-07,2.46316e-06,6.25204e-07,3.29331e-08,6.04426e-09,5.57486e-09,2.27543e-08,4.3403e-07,3.87094e-07,1.01482e-07,4.03867e-07,...,3.60153e-09,3.78887e-09,3.59061e-09,3.21305e-09,3.44564e-09,3.17323e-09,3.08611e-09,4.48209e-09,4.57867e-09,3.43811e-09,3.62932e-09,3.96026e-09,6.33567e-09,4.14e-09,5.69481e-09,3.43762e-07,1.97797e-07,3.28433e-09,3.30347e-09,7.67532e-08,6.63836e-07,5.08731e-07,6.77146e-07,1.25409e-06,1.40929e-06,2.17827e-06,5.32421e-06,1.5735e-06,5.2268e-07,3.5174e-07,1.53445e-07,1.71656e-06,7.47147e-07,5.59135e-08,5.28861e-08,8.72358e-09,3.30508e-09,3.39847e-09,3.53452e-09,3.02045e-09
545775,0.0153009,0.0410511,0.0203779,0.00176169,0.00143707,0.0110665,0.0626091,0.08957,0.0152084,0.000740258,0.00194512,0.00957221,0.0035683,0.000429786,0.000423763,0.00792726,0.0244004,0.00586593,0.000574236,3.95829e-05,0.00100119,0.00962013,0.0247482,0.0256188,0.0165123,0.0327513,0.0766023,0.0320967,0.00302803,7.79097e-05,9.64102e-06,3.59757e-06,2.72465e-06,2.33583e-06,2.29765e-06,2.39196e-06,2.32353e-06,2.30919e-06,2.34423e-06,2.38468e-06,...,1.76953,1.59214,1.07827,0.595891,0.594308,0.37076,0.553333,0.365506,0.255991,0.138859,0.114033,0.0957489,0.0855866,0.117575,0.154242,0.0924141,0.0517471,0.0214908,0.0198752,0.0119293,0.0154542,0.00796854,0.00398772,0.00419107,0.00374636,0.00246336,0.00170943,0.00130506,0.000551166,0.000534649,0.000314366,0.000127135,0.000104077,2.50876e-05,2.72643e-05,1.05538e-05,1.071e-05,4.56655e-06,4.51945e-06,4.42554e-06
540857,1.01472e-08,9.286e-09,9.24723e-09,9.63285e-09,9.18616e-09,9.39508e-09,9.14915e-09,8.42323e-09,9.10948e-09,9.0798e-09,1.26257e-08,1.9737e-07,2.0133e-06,2.10217e-06,9.64599e-06,9.51892e-06,2.64058e-06,2.09171e-06,3.55123e-05,0.000216854,0.000419188,0.000365759,0.00018954,0.00015035,0.000210375,0.000236178,0.000166975,8.6581e-05,3.5308e-05,4.82448e-05,0.000159228,0.000336574,0.000137993,5.8594e-05,5.89675e-05,0.000137312,0.00022144,0.000129463,0.000153022,0.000175689,...,3.24323e-05,1.52913e-05,3.6286e-05,6.26623e-05,0.000128456,5.73143e-05,3.04376e-05,1.19481e-05,3.57273e-05,4.69906e-05,6.22029e-05,7.526e-05,6.99079e-05,9.35467e-05,4.23019e-05,3.6467e-07,6.44388e-07,2.88163e-09,2.86768e-09,2.64299e-09,2.27427e-08,1.89994e-07,1.5834e-07,2.43265e-07,1.48829e-07,1.09097e-06,5.43741e-08,2.5254e-09,3.1793e-09,9.10786e-08,1.13964e-07,1.43705e-07,5.97289e-08,2.55945e-09,2.75409e-09,2.84289e-09,2.82248e-09,2.5688e-09,2.54676e-09,2.86047e-09
540855,7.58787e-09,7.47717e-09,7.37175e-09,6.73093e-09,6.71347e-09,7.32788e-09,7.03995e-09,6.71042e-09,7.10086e-09,7.26497e-09,7.00687e-09,6.96691e-09,6.55929e-09,6.75426e-09,7.94797e-09,8.75073e-09,8.83849e-08,9.30482e-07,2.94796e-06,2.17131e-06,3.05536e-06,1.07579e-06,7.01531e-08,1.42526e-08,1.58637e-07,1.60501e-06

In [9]:
def create_df_umap(df1, df2, df3):

    df_umap_step1 = df1.merge(df2, how ='left', left_index=True, right_index=True)
    df_umap_step2 = df_umap_step1.merge(df3, how ='left', left_index=True, right_index=True)
    return df_umap_step2


df_umap = create_df_umap(dataset, df_mfcc, df_mel)

if run_on_colab == True:
        outfile = '/content/content/MyDrive/bird-songs/df_umap.pkl'
else:
    outfile = './mels_8s_arrays.npy'

df_umap.to_pickle(outfile)

display(df_umap)

,label,mfcc_avg1,mfcc_avg2,mfcc_avg3,mfcc_avg4,mfcc_avg5,mfcc_avg6,mfcc_avg7,mfcc_avg8,mfcc_avg9,mfcc_avg10,mfcc_avg11,mfcc_avg12,mfcc_avg13,mfcc_avg14,mfcc_avg15,mfcc_avg16,mfcc_avg17,mfcc_avg18,mfcc_avg19,mfcc_avg20,mfcc_avg21,mfcc_avg22,mfcc_avg23,mfcc_avg24,mfcc_std1,mfcc_std2,mfcc_std3,mfcc_std4,mfcc_std5,mfcc_std6,mfcc_std7,mfcc_std8,mfcc_std9,mfcc_std10,mfcc_std11,mfcc_std12,mfcc_std13,mfcc_std14,mfcc_std15,...,mel_std89,mel_std90,mel_std91,mel_std92,mel_std93,mel_std94,mel_std95,mel_std96,mel_std97,mel_std98,mel_std99,mel_std100,mel_std101,mel_std102,mel_std103,mel_std104,mel_std105,mel_std106,mel_std107,mel_std108,mel_std109,mel_std110,mel_std111,mel_std112,mel_std113,mel_std114,mel_std115,mel_std116,mel_std117,mel_std118,mel_std119,mel_std120,mel_std121,mel_std122,mel_std123,mel_std124,mel_std125,mel_std126,mel_std127,mel_std128
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
564895,3,-889.492,5.0335,-31.4773,4.73073,22.9555,-11.6972,-14.5283,10.5064,-0.979624,-6.92005,3.75479,5.51634,-1.98331,4.45234,1.58094,-6.79015,0.780834,4.96603,-3.84318,-3.38185,3.86442,0.416703,-3.63969,1.73831,54.8656,20.6248,61.4001,23.5236,43.6314,28.4721,24.2452,27.3742,11.3261,21.4315,12.8862,15.7766,11.2154,11.5864,7.85349,...,3.60153e-09,3.78887e-09,3.59061e-09,3.21305e-09,3.44564e-09,3.17323e-09,3.08611e-09,4.48209e-09,4.57867e-09,3.43811e-09,3.62932e-09,3.96026e-09,6.33567e-09,4.14e-09,5.69481e-09,3.43762e-07,1.97797e-07,3.28433e-09,3.30347e-09,7.67532e-08,6.63836e-07,5.08731e-07,6.77146e-07,1.25409e-06,1.40929e-06,2.17827e-06,5.32421e-06,1.5735e-06,5.2268e-07,3.5174e-07,1.53445e-07,1.71656e-06,7.47147e-07,5.59135e-08,5.28861e-08,8.72358e-09,3.30508e-09,3.39847e-09,3.53452e-09,3.02045e-09
545775,3,-629.473,-44.4655,-47.775,61.9831,-1.03335,-0.152016,-10.5474,2.96929,1.58006,10.9912,4.00956,-1.31302,3.10983,0.962198,-0.161864,0.784014,-1.68879,-7.63529,3.80711,4.49119,-2.39815,-2.61737,2.86433,1.15802,144.486,76.5706,76.4662,70.7324,20.5744,22.7741,32.5819,14.9177,13.7567,20.6891,17.6878,13.1438,12.5093,13.144,15.8983,...,1.76953,1.59214,1.07827,0.595891,0.594308,0.37076,0.553333,0.365506,0.255991,0.138859,0.114033,0.0957489,0.0855866,0.117575,0.154242,0.0924141,0.0517471,0.0214908,0.0198752,0.0119293,0.0154542,0.00796854,0.00398772,0.00419107,0.00374636,0.00246336,0.00170943,0.00130506,0.000551166,0.000534649,0.000314366,0.000127135,0.000104077,2.50876e-05,2.72643e-05,1.05538e-05,1.071e-05,4.56655e-06,4.51945e-06,4.42554e-06
540857,3,-837.42,20.6867,-56.4215,-14.9488,2.62786,-2.47709,-3.9447,-4.24075,-2.73174,-1.02478,6.68777,-0.485857,-1.94842,13.7119,6.50528,-4.9148,-3.66503,-2.16639,-1.29081,-3.2789,-4.2712,-1.3895,4.98414,2.9093,69.8873,41.4715,57.9993,51.3175,43.4628,34.1108,25.187,30.4362,18.6808,20.153,17.7549,20.3741,18.7521,20.6494,18.654,...,3.24323e-05,1.52913e-05,3.6286e-05,6.26623e-05,0.000128456,5.73143e-05,3.04376e-05,1.19481e-05,3.57273e-05,4.69906e-05,6.22029e-05,7.526e-05,6.99079e-05,9.35467e-05,4.23019e-05,3.6467e-07,6.44388e-07,2.88163e-09,2.86768e-09,2.64299e-09,2.27427e-08,1.89994e-07,1.5834e-07,2.43265e-07,1.48829e-07,1.09097e-06,5.43741e-08,2.5254e-09,3.1793e-09,9.10786e-08,1.13964e-07,1.43705e-07,5.97289e-08,2.55945e-09,2.75409e-09,2.84289e-09,2.82248e-09,2.5688e-09,2.54676e-09,2.86047e-09
540855,3,-812.35,-12.1207,-45.2474,3.26925,11.5764,1.21442,-3.43979,4.12308,-5.66603,5.67809,-3.74287,-0.625781,4.12183,-4.8242,1.30128,-1.2018,7.83683,0.461579,-8.2654,1.14232,0.390527,0.0296896,0.416522,4.08837,83.4465,28.7416,80.8335,35.1697,39.1032,27.3928,19.5737,21.4965,18.8731,12.5815,14.7667,14.2993,13.5988,13.3202,17.7982,...,0.00468365,0.00231512,0.0041563,0.00199671,0.000709183,0.000361381,0.000282941,8.13725e-05,7.66919e-05,3.32326e-05,1.00695e-05,6.20375e-06,9.37737e-06,1.04607e-05,9.88565e-06,9.98848e-06,4.40706e-06,1.05517e-05,3.49421e-05,9.6591e-05,2.00168e-05,1.74487e-05,1.09331e-05,1.70521e-05,2.0919e-05,1.39387e-05,2.7442e-05,9.55492e-07,1.03718e-06,5.73188e-07,6.11957e

In [6]:
def create_umap_chart(df, num_features=20, color_feature='label'):
    
    umap_2d = UMAP(random_state=0)
    umap_2d.fit(df.iloc[:, -num_features:].values)
    
    projections = umap_2d.transform(df.iloc[:, -num_features:].values)
    
    fig = px.scatter(
        projections, x=0, y=1,
        color=list(df[color_feature].astype(str)), labels={'color': color_feature},
        title = f"UMAP Plot of bird audio samples colored by {color_feature}"
    )
    
    return fig


num_mfcc_features = 24*2
num_mel_features = 128*2
num_mfcc_and_mel_features = num_mfcc_features + num_mel_features
print(f"Total number of features is {num_mfcc_and_mel_features}")
#print(f"Total number of bird audio samples is {len(df_umap)}")

if run_on_colab == True:
        infile = '/content/content/MyDrive/bird-songs/df_umap.pkl'
else:
    infile = './df_umap.pkl'

df_umap = pd.read_pickle(infile)


create_umap_chart(df_umap, num_features=num_mfcc_and_mel_features, color_feature='label')


Total number of features is 304
